# Price Impact Analysis (Case: 5bus_nrel)

In this analysis, we want to study the structure of Price Impact / Price formation in electricity markets.

## Dependencies and Data Preparation

#### Packages and Diretory

In [57]:
using Pkg
Pkg.activate("./examples/5bus_nrel/price_impact_analysis_5bus_nrel")

In [58]:
Pkg.instantiate()

In [54]:
cd("C:/Users/55329/Desktop/Projeto/GridAnalysis.jl")

In [60]:
pwd()

"C:\\Users\\55329\\Desktop\\Projeto\\GridAnalysis.jl"

In [59]:
using Dates
using DataFrames
using GridAnalysis
using Gurobi
using PowerSystems
using PowerSimulations
using Test
using Measures
using Plots

#### Uploading Utils

Might not work if running lines manually 
(solution: edit to be the path for this examples directory for example: 'example_dir = "./examples/5bus_nrel/"')

In [61]:
example_dir = "./examples/5bus_nrel/"
data_dir = joinpath(example_dir, "data")
include(joinpath(example_dir, "utils.jl"))

plot_DA_RT

#### Data Prep and Build Market Simulator
define solvers for Unit Commitment (UC), Real Time (RT) and Economic Dispatch (ED)

In [62]:
solver_uc = optimizer_with_attributes(Gurobi.Optimizer)
solver_rt = optimizer_with_attributes(Gurobi.Optimizer)
solver_ed = optimizer_with_attributes(Gurobi.Optimizer)

MathOptInterface.OptimizerWithAttributes(Gurobi.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[])

## Define analysis to be done

In [143]:
Network = [DCPPowerModel];
Ramp = [true];
Minimal_generation = [true];
Reserve = [true];
Offer_Bus = ["bus1", "bus2", "bus3", "bus4", "bus5"];
bidding_period = [collect(1:24), [1]];

#### Create a DataFrame where each line represents one analysis 

In [144]:
df = DataFrame(Network = [], 
               Ramp = [],
               Minimal_generation = [],
               Reserve = [],
               Offer_Bus = [],
               bidding_period = []) 

for a in Network 
    for b in Ramp 
        for c in Minimal_generation 
            for d in Reserve
                for e in Offer_Bus
                    for f in bidding_period
                        new_data = DataFrame(Network = [a], 
                        Ramp = [b],
                        Minimal_generation = [c],
                        Reserve = [d],
                        Offer_Bus = [e],
                        bidding_period = [f])
                        append!(df, new_data)
                    end
                end
            end
        end
    end
end

In [145]:
df

,Network,Ramp,Minimal_generation,Reserve,Offer_Bus,bidding_period
,Any,Any,Any,Any,Any,Any
1,DCPPowerModel,1,1,1,bus1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]"
2,DCPPowerModel,1,1,1,bus1,[1]
3,DCPPowerModel,1,1,1,bus2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]"
4,DCPPowerModel,1,1,1,bus2,[1]
5,DCPPowerModel,1,1,1,bus3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]"
6,DCPPowerModel,1,1,1,bus3,[1]
7,DCPPowerModel,1,1,1,bus4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]"
8,DCPPowerModel,1,1,1,bus4,[1]
9,DCPPowerModel,1,1,1,bus5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]"


#### Define range quota and period to be analysed

In [146]:
range_quota = Float64.(collect(0:0.1:4));
period_analysed = [5,19];

lin, col = size(df)

(10, 6)

## Run all the simulations

TODO: Make it a function

TODO: Don't do the simulation again if it is already in the diretory

TODO: Add changes to Ramp and Minimal_generation if it is false

In [147]:
df.bidding_period[2]

1-element Vector{Int64}:
 1

In [148]:
for l=2:lin
    # call our data preparation to build base system
    # the case was modified to not have hydros nor transformers
    sys_rt = build_5_bus_matpower_RT(data_dir;)

    base_da_system = build_5_bus_matpower_DA(
        data_dir;
        # using a modified (mod) file that reduced load for feasibility in DC-OPF
        forecasts_pointers_file=joinpath(
            data_dir, "forecasts", "timeseries_pointers_da_7day_mod.json"
        ),
        add_reserves=df.Reserve[l],
    )
    

    # Add single generator at a defined bus
    gen = add_gerator!(base_da_system, df.Offer_Bus[l], (min=0.0, max=0.0))
    @test gen in get_components(Generator, base_da_system)

    # create and set variable cost time-series for the generator
    ts_array = create_generator_bids(;
        initial_bidding_time=DateTime("2020-01-01"),
        bidding_periods=df.bidding_period[l],
        system=base_da_system,
        costs=zeros(length(df.bidding_period[l])),
    )
    set_variable_cost!(base_da_system, gen, ts_array)
    
    # duplicate system and prepare times series for the time varying parameters (loads, renewables, ...)
    sys_uc, sys_ed = prep_systems_UCED(base_da_system)

    # generic market formulation templates with defined network formulation
    # CopperPlate-OPF: network=CopperPlatePowerModel
    # DC-OPF: network=DCPPowerModel
    # NFA-OPF (only line limit constraints): network=NFAPowerModel
    # DC-PTDF-OPF (what ISOs do): network=StandardPTDFModel
    template_uc = template_unit_commitment(; network=df.Network[l])
    template_rt = template_economic_dispatch(; network=df.Network[l])
    template_ed = template_economic_dispatch(; network=df.Network[l])

    # build a market clearing simulator
    market_simulator = UCEDRT(;
        system_uc=sys_uc,
        system_rt=sys_rt,
        system_ed=sys_ed,
        template_uc=template_uc,
        template_rt=template_rt,
        template_ed=template_ed,
        solver_uc=solver_uc,
        solver_rt=solver_rt,
        solver_ed=solver_ed,
    )

    @test isa(market_simulator, UCEDRT)

    #Calculates the dispatch result for a bid curve
    name_generator = get_name(gen);
    initial_time = Date("2020-01-01");
    steps = 1;

    directory_name= "Network_" * string(df.Network[l]) * "__Ramp_" * string(df.Ramp[l]) * 
    "__Min_gen_" * string(df.Minimal_generation[l]) * "__Reserve_" * string(df.Reserve[l]) *
    "__Offer_Bus_" * string(df.Offer_Bus[l]) * "__bidding_period_1-" * string(length(df.bidding_period[l]))
    
    #if doesnt exists the diretory;
    mkdir(joinpath(example_dir, "price_impact_analysis_5bus_nrel/results", directory_name))
    simulation_folder = joinpath(example_dir, "price_impact_analysis_5bus_nrel/results", directory_name)

    lmps_df, results_df = pq_curves_virtuals!(
        market_simulator, name_generator, range_quota, initial_time, steps, simulation_folder
    )

end

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\55329\.julia\packages\ProgressMeter\l7LEt\src\ProgressMeter.jl:593
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
  Step:                  1
  problem:               RT
  Simulation Timestamp:  2020-01-01T23:55:00
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-1\rt_quota_4.0\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390


### Locality Impact: What is the impact of the virtual offer as we move away from the offer node? 

Analytics metrics to be used:

1. Price change for nodes connected and nodes not connected to the offer node;
2. Price change according to the distance to the offer node. Where "distance" refers to the number of buses between them;
3. Price change according to the bottleneck between the buses and the offer node. Where "bottleneck" is related to the trasmission lines capacity limits.

##### Load the simulation done previously 

In [160]:
lines=[1,3,5,7,9]
period_analysed=[[5],[19]] 

2-element Vector{Vector{Int64}}:
 [5]
 [19]

In [161]:
plt_h=Array{Any}(nothing, length(lines), length(period_analysed))

for (x,l) in enumerate(lines)
    
    # call our data preparation to build base system
    # the case was modified to not have hydros nor transformers
    sys_rt = build_5_bus_matpower_RT(data_dir;)

    base_da_system = build_5_bus_matpower_DA(
        data_dir;
        # using a modified (mod) file that reduced load for feasibility in DC-OPF
        forecasts_pointers_file=joinpath(
            data_dir, "forecasts", "timeseries_pointers_da_7day_mod.json"
        ),
        add_reserves=df.Reserve[l],
    )

    # Add single generator at a defined bus
    gen = add_gerator!(base_da_system, df.Offer_Bus[l], (min=0.0, max=0.0))
    @test gen in get_components(Generator, base_da_system)

    # create and set variable cost time-series for the generator
    ts_array = create_generator_bids(;
        initial_bidding_time=DateTime("2020-01-01"),
        bidding_periods=df.bidding_period[l],
        system=base_da_system,
        costs=zeros(length(df.bidding_period[l])),
    )
    set_variable_cost!(base_da_system, gen, ts_array)

    # duplicate system and prepare times series for the time varying parameters (loads, renewables, ...)
    sys_uc, sys_ed = prep_systems_UCED(base_da_system)

    # generic market formulation templates with defined network formulation
    # CopperPlate-OPF: network=CopperPlatePowerModel
    # DC-OPF: network=DCPPowerModel
    # NFA-OPF (only line limit constraints): network=NFAPowerModel
    # DC-PTDF-OPF (what ISOs do): network=StandardPTDFModel
    template_uc = template_unit_commitment(; network=df.Network[l])
    template_rt = template_economic_dispatch(; network=df.Network[l])
    template_ed = template_economic_dispatch(; network=df.Network[l])

    # build a market clearing simulator
    market_simulator = UCEDRT(;
        system_uc=sys_uc,
        system_rt=sys_rt,
        system_ed=sys_ed,
        template_uc=template_uc,
        template_rt=template_rt,
        template_ed=template_ed,
        solver_uc=solver_uc,
        solver_rt=solver_rt,
        solver_ed=solver_ed,
    )

    @test isa(market_simulator, UCEDRT)

    directory_name= "Network_" * string(df.Network[l]) * "__Ramp_" * string(df.Ramp[l]) * 
    "__Min_gen_" * string(df.Minimal_generation[l]) * "__Reserve_" * string(df.Reserve[l]) *
    "__Offer_Bus_" * string(df.Offer_Bus[l]) * "__bidding_period_1-" * string(length(df.bidding_period[l]))

    simulation_folder = joinpath(example_dir, "price_impact_analysis_5bus_nrel/results", directory_name)

    lmps_df, results_df = load_pq_curves(market_simulator, range_quota, simulation_folder)

    @test isa(results_df[range_quota[1]], Dict{String,SimulationResults})
    @test isa(lmps_df[range_quota[1]], Dict{String,DataFrame})
    
    for (y,t) in enumerate(period_analysed)
        plt_h[x,y] = plot_price_curves(lmps_df, period_analysed[y], unique(df.Offer_Bus), df.Offer_Bus[l], initial_time)
    end
end

┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\parsers\pm_io\matpower.jl:362
┌ Info: extending matpower format with data: gen_name 7x4
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\parsers\pm_io\matpower.jl:362
┌ Info: extending matpower format by appending matrix "gen_name" in to "gen"
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\parsers\pm_io\matpower.jl:703
┌ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\parsers\pm_io\data.jl:1221
┌ Info: Constructing System from Power Models
│   data["name"] = nesta_case5_pjm
│   data["source_type"] = matpower
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\parsers\power_models_data.jl:52
┌ Info: Reading bus data
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W

        1 arquivo(s) copiado(s).


┌ Info: Serialized time series data to C:\Users\55329\AppData\Local\Temp\jl_jaGQkb\sys_time_series_storage.h5.
└ @ InfrastructureSystems C:\Users\55329\.julia\packages\InfrastructureSystems\lnBOX\src\time_series_storage.jl:97


        1 arquivo(s) copiado(s).


┌ Info: Serialized System to C:\Users\55329\AppData\Local\Temp\jl_jaGQkb\sys.json
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\base.jl:258
┌ Info: Loaded time series from storage file existing=sys_time_series_storage.h5 new=C:\Users\55329\AppData\Local\Temp\jl_1E12.tmp compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems C:\Users\55329\.julia\packages\InfrastructureSystems\lnBOX\src\hdf5_time_series_storage.jl:98
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus1__bidding_period_1-24\da_quota_0.0\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus1__bidding_pe

...

        1 arquivo(s) copiado(s).


┌ Info: Serialized System to C:\Users\55329\AppData\Local\Temp\jl_j24Vno\sys.json
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\base.jl:258
┌ Info: Loaded time series from storage file existing=sys_time_series_storage.h5 new=C:\Users\55329\AppData\Local\Temp\jl_4E17.tmp compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems C:\Users\55329\.julia\packages\InfrastructureSystems\lnBOX\src\hdf5_time_series_storage.jl:98
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-24\da_quota_0.0\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_pe

┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-24\da_quota_1.2\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-24\rt_quota_1.2\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-24\da_quota_1.3\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking in

┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-24\da_quota_2.5\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-24\rt_quota_2.5\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-24\da_quota_2.6\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking in

┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-24\da_quota_3.8\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-24\rt_quota_3.8\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-24\da_quota_3.9\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking in

#### Results

In [167]:
plot(plt[1,1],plt[1,2],layout=(2,1))

<!DOCTYPE html>
 
 
 Plots.jl

In [168]:
plot(plt[2,1],plt[2,2],layout=(2,1))

<!DOCTYPE html>
 
 
 Plots.jl

In [169]:
plot(plt[3,1],plt[3,2],layout=(2,1))

<!DOCTYPE html>
 
 
 Plots.jl

In [170]:
plot(plt[4,1],plt[4,2],layout=(2,1))

<!DOCTYPE html>
 
 
 Plots.jl

In [171]:
plot(plt[5,1],plt[5,2],layout=(2,1))

<!DOCTYPE html>
 
 
 Plots.jl

### Hour Impact: What is the impact of the virtual offer as we move away from the offer period?
Proposed evaluation experiment: Make an offer in the first hour and see the impact in the other hours.

#### Load the simulation done previously

In [172]:
lines=[2,4,6,8,10]
period_analysed=[[5],[19]] 

2-element Vector{Vector{Int64}}:
 [5]
 [19]

In [174]:
plt_h=Array{Any}(nothing, length(lines), length(period_analysed))

for (x,l) in enumerate(lines)
    
    # call our data preparation to build base system
    # the case was modified to not have hydros nor transformers
    sys_rt = build_5_bus_matpower_RT(data_dir;)

    base_da_system = build_5_bus_matpower_DA(
        data_dir;
        # using a modified (mod) file that reduced load for feasibility in DC-OPF
        forecasts_pointers_file=joinpath(
            data_dir, "forecasts", "timeseries_pointers_da_7day_mod.json"
        ),
        add_reserves=df.Reserve[l],
    )

    # Add single generator at a defined bus
    gen = add_gerator!(base_da_system, df.Offer_Bus[l], (min=0.0, max=0.0))
    @test gen in get_components(Generator, base_da_system)

    # create and set variable cost time-series for the generator
    ts_array = create_generator_bids(;
        initial_bidding_time=DateTime("2020-01-01"),
        bidding_periods=df.bidding_period[l],
        system=base_da_system,
        costs=zeros(length(df.bidding_period[l])),
    )
    set_variable_cost!(base_da_system, gen, ts_array)

    # duplicate system and prepare times series for the time varying parameters (loads, renewables, ...)
    sys_uc, sys_ed = prep_systems_UCED(base_da_system)

    # generic market formulation templates with defined network formulation
    # CopperPlate-OPF: network=CopperPlatePowerModel
    # DC-OPF: network=DCPPowerModel
    # NFA-OPF (only line limit constraints): network=NFAPowerModel
    # DC-PTDF-OPF (what ISOs do): network=StandardPTDFModel
    template_uc = template_unit_commitment(; network=df.Network[l])
    template_rt = template_economic_dispatch(; network=df.Network[l])
    template_ed = template_economic_dispatch(; network=df.Network[l])

    # build a market clearing simulator
    market_simulator = UCEDRT(;
        system_uc=sys_uc,
        system_rt=sys_rt,
        system_ed=sys_ed,
        template_uc=template_uc,
        template_rt=template_rt,
        template_ed=template_ed,
        solver_uc=solver_uc,
        solver_rt=solver_rt,
        solver_ed=solver_ed,
    )

    @test isa(market_simulator, UCEDRT)

    directory_name= "Network_" * string(df.Network[l]) * "__Ramp_" * string(df.Ramp[l]) * 
    "__Min_gen_" * string(df.Minimal_generation[l]) * "__Reserve_" * string(df.Reserve[l]) *
    "__Offer_Bus_" * string(df.Offer_Bus[l]) * "__bidding_period_1-" * string(length(df.bidding_period[l]))

    simulation_folder = joinpath(example_dir, "price_impact_analysis_5bus_nrel/results", directory_name)

    lmps_df, results_df = load_pq_curves(market_simulator, range_quota, simulation_folder)

    @test isa(results_df[range_quota[1]], Dict{String,SimulationResults})
    @test isa(lmps_df[range_quota[1]], Dict{String,DataFrame})
    
    for (y,t) in enumerate(period_analysed)
        plt_h[x,y] = plot_price_curves(lmps_df, period_analysed[y], unique(df.Offer_Bus), df.Offer_Bus[l], initial_time)
    end
end

┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\parsers\pm_io\matpower.jl:362
┌ Info: extending matpower format with data: gen_name 7x4
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\parsers\pm_io\matpower.jl:362
┌ Info: extending matpower format by appending matrix "gen_name" in to "gen"
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\parsers\pm_io\matpower.jl:703
┌ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\parsers\pm_io\data.jl:1221
┌ Info: Constructing System from Power Models
│   data["name"] = nesta_case5_pjm
│   data["source_type"] = matpower
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\parsers\power_models_data.jl:52
┌ Info: Reading bus data
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W

        1 arquivo(s) copiado(s).


┌ Info: Serialized time series data to C:\Users\55329\AppData\Local\Temp\jl_tNOiYL\sys_time_series_storage.h5.
└ @ InfrastructureSystems C:\Users\55329\.julia\packages\InfrastructureSystems\lnBOX\src\time_series_storage.jl:97


        1 arquivo(s) copiado(s).


┌ Info: Serialized System to C:\Users\55329\AppData\Local\Temp\jl_tNOiYL\sys.json
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\base.jl:258
┌ Info: Loaded time series from storage file existing=sys_time_series_storage.h5 new=C:\Users\55329\AppData\Local\Temp\jl_BA12.tmp compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems C:\Users\55329\.julia\packages\InfrastructureSystems\lnBOX\src\hdf5_time_series_storage.jl:98
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus1__bidding_period_1-1\da_quota_0.0\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus1__bidding_per

...

┌ Info: Reading storage data
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\parsers\power_models_data.jl:707
┌ Info: Serialized time series data to C:\Users\55329\AppData\Local\Temp\jl_hJv0VX\sys_time_series_storage.h5.
└ @ InfrastructureSystems C:\Users\55329\.julia\packages\InfrastructureSystems\lnBOX\src\time_series_storage.jl:97


        1 arquivo(s) copiado(s).


┌ Info: Serialized System to C:\Users\55329\AppData\Local\Temp\jl_hJv0VX\sys.json
└ @ PowerSystems C:\Users\55329\.julia\packages\PowerSystems\DdZ5W\src\base.jl:258
┌ Info: Loaded time series from storage file existing=sys_time_series_storage.h5 new=C:\Users\55329\AppData\Local\Temp\jl_3D87.tmp compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems C:\Users\55329\.julia\packages\InfrastructureSystems\lnBOX\src\hdf5_time_series_storage.jl:98
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-1\da_quota_0.0\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_per

┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-1\rt_quota_1.2\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-1\da_quota_1.3\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-1\rt_quota_1.3\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integ

┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-1\da_quota_2.7\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-1\rt_quota_2.7\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integrity of ./examples/5bus_nrel/price_impact_analysis_5bus_nrel/results\Network_DCPPowerModel__Ramp_true__Min_gen_true__Reserve_true__Offer_Bus_bus5__bidding_period_1-1\da_quota_2.8\1\data_store\simulation_store.h5
└ @ PowerSimulations C:\Users\55329\.julia\packages\PowerSimulations\0onip\src\utils.jl:390
┌ Info: checking integ

#### Results

In [175]:
plot(plt_h[1,1],plt_h[1,2],layout=(2,1))

<!DOCTYPE html>
 
 
 Plots.jl

In [176]:
plot(plt_h[2,1],plt_h[2,2],layout=(2,1))

<!DOCTYPE html>
 
 
 Plots.jl

In [177]:
plot(plt_h[3,1],plt_h[3,2],layout=(2,1))

<!DOCTYPE html>
 
 
 Plots.jl

In [178]:
plot(plt_h[4,1],plt_h[4,2],layout=(2,1))

<!DOCTYPE html>
 
 
 Plots.jl

In [179]:
plot(plt_h[5,1],plt_h[5,2],layout=(2,1))

<!DOCTYPE html>
 
 
 Plots.jl